# 코딩 전 생각

In [ ]:
""""""""""
아기 상어 문제보다 조금 더 난이도가 있는 문제인 것 같다.

문제에서 요구한 대로 구현하되, 먹은 물고시 수의 합이 최대가 되어야 하므로,

더 이상 먹을 물고기가 없을 때마다 지금까지 먹은 물고기 번호의 합의 최댓값을 갱신하는 DFS를 이용하면 될 것 같다.

""""""""""

# 내 답안

In [49]:
import heapq
import copy

graph = [[] for _ in range(4)]
for i in range(4):
    data = list(map(int, input().split()))
    for j in range(0, 7, 2):
        graph[i].append([data[j], data[j+1]])
        
def move(now_x, now_y):
    global graph
    q = []
    for i in range(4):
        for j in range(4):
            if graph[i][j] != None:
                heapq.heappush(q, (graph[i][j][0], graph[i][j][1]))
    while q:
        now = heapq.heappop(q)
        a, b = now
        i, j  = find_location(a, b, graph)
        for _ in range(8):  
            nx = i + dx[b]
            ny = j + dy[b]
            if 0 <= nx < 4 and 0 <= ny < 4 and (nx,ny) != (now_x, now_y):
                if graph[nx][ny] == None:
                    graph[nx][ny] = [a, b]
                    graph[i][j] = None
                else:
                    graph[nx][ny], graph[i][j] = graph[i][j], graph[nx][ny]
                break

            else:
                if b == 8:
                    b = 1
                    graph[i][j][1] = 1
                    
                else:
                    b += 1
                    graph[i][j][1] += 1
        
def find_location(a, b, graph):
    for i in range(4):
        for j in range(4):
            if graph[i][j] == [a, b]:
                return (i, j)

def hunt(now_x, now_y, now_direction):
    result = []
    global graph
    while True:
        nx = now_x + dx[now_direction]
        ny = now_y + dy[now_direction]
        if nx < 0 or nx >= 4 or ny < 0 or ny >= 4:
            break
        if graph[nx][ny] != None:
            fish = graph[nx][ny]
            result.append((nx, ny, fish[0], fish[1]))
        now_x , now_y = nx, ny
    return result

dx = [0, -1, -1, 0, 1, 1, 1, 0, -1]
dy = [0, 0, -1, -1, -1, 0, 1, 1, 1]
now_x, now_y = 0,0
now_direction = graph[0][0][1]
ate = 0
ate = graph[0][0][0]
graph[0][0] = None
max_value = -1e9

def process(now_x, now_y, now_direction):
    global max_value
    global ate
    global graph
    move(now_x, now_y)
    x = now_x
    y = now_y
    r = now_direction
    g = copy.deepcopy(graph)
    possible = hunt(now_x, now_y, now_direction)
    if len(possible) == 0:
        max_value = max(ate, max_value)
        return

    for p in possible:
        a, b, c, d = p
        now_x, now_y = a, b
        ate += c
        now_direction = d
        graph[a][b] = None
        process(now_x, now_y, now_direction)
        now_x, now_y = x, y
        ate -= c
        now_direction = r
        graph = g

process(0, 0, now_direction)
print(max_value)

7 1 2 6 15 7 9 3
3 5 1 4 14 1 10 6
6 4 13 3 4 6 11 1
16 5 8 7 5 2 12 2
7


# 답안 작성 후 생각

In [ ]:
"""""""""
문제 자체를 이해하는 것은 어렵지 않았으나, 구현이 까다로웠다.

특히 입력 조건을 설정할 때 인덱스 오류가 잦았다.

게다가 이중 리스트의 복사를 list(list이름)으로 했는데, 전과 다르게 정상 작동하지 않는 것을 발견하는 데까지 오랜 시간이 걸렸다.

주어진 입력 예시 3가지와 백준의 추가 테케 3가지를 모두 시도해본 결과 정답을 출력하지만,

무슨 이유에서인지 백준 채점기로는 오답을 기록한다.
"""""""""

# 정답

In [ ]:
import copy

# 4 X 4 크기 격자에 존재하는 각 물고기의 번호(없으면 -1)와 방향 값을 담는 테이블
array = [[None] * 4 for _ in range(4)]

for i in range(4):
    data = list(map(int, input().split()))
    # 매 줄마다 4마리의 물고기를 하나씩 확인하며
    for j in range(4):
        # 각 위치마다 [물고기의 번호, 방향]을 저장
        array[i][j] = [data[j * 2], data[j * 2 + 1] - 1]

# 8가지 방향에 대한 정의
dx = [-1, -1, 0, 1, 1, 1, 0, -1]
dy = [0, -1, -1, -1, 0, 1, 1, 1]

# 현재 위치에서 왼쪽으로 회전된 결과 반환
def turn_left(direction):
    return (direction + 1) % 8

result = 0 # 최종 결과

# 현재 배열에서 특정한 번호의 물고기 위치 찾기
def find_fish(array, index):
    for i in range(4):
        for j in range(4):
            if array[i][j][0] == index:
                return (i, j)
    return None

# 모든 물고기를 회전 및 이동시키는 함수
def move_all_fishes(array, now_x, now_y):
    # 1번부터 16번까지의 물고기를 차례대로 (낮은 번호부터) 확인
    for i in range(1, 17):
        # 해당 물고기의 위치를 찾기
        position = find_fish(array, i)
        if position != None:
            x, y = position[0], position[1]
            direction = array[x][y][1]
            # 해당 물고기의 방향을 왼쪽으로 계속 회전시키며 이동이 가능한지 확인
            for j in range(8):
                nx = x + dx[direction]
                ny = y + dy[direction]
                # 해당 방향으로 이동이 가능하다면 이동 시키기
                if 0 <= nx and nx < 4 and 0 <= ny and ny < 4:
                    if not (nx == now_x and ny == now_y):
                        array[x][y][1] = direction
                        array[x][y], array[nx][ny] = array[nx][ny], array[x][y]
                        break
                direction = turn_left(direction)
        
# 상어가 현재 위치에서 먹을 수 있는 모든 물고기의 위치 반환
def get_possible_positions(array, now_x, now_y):
    positions = []
    direction = array[now_x][now_y][1]
    # 현재의 방향으로 쭉 이동하기
    for i in range(4):
        now_x += dx[direction]
        now_y += dy[direction]
        # 범위를 벗어나지 않는지 확인하며
        if 0 <= now_x and now_x < 4 and 0 <= now_y and now_y < 4:
            # 물고기가 존재하는 경우
            if array[now_x][now_y][0] != -1:
                positions.append((now_x, now_y))
    return positions

# 모든 경우를 탐색하기 위한 DFS 함수
def dfs(array, now_x, now_y, total):
    global result
    array = copy.deepcopy(array) # 리스트를 통째로 복사
    
    total += array[now_x][now_y][0] # 현재 위치의 물고기 먹기
    array[now_x][now_y][0] = -1 # 물고기를 먹었으므로 번호 값을 -1로 변환
    
    move_all_fishes(array, now_x, now_y) # 전체 물고기 이동 시키기

    # 이제 다시 상어가 이동할 차례이므로, 이동 가능한 위치 찾기
    positions = get_possible_positions(array, now_x, now_y)
    # 이동할 수 있는 위치가 하나도 없다면 종료
    if len(positions) == 0:
        result = max(result, total) # 최댓값 저장
        return 
    # 모든 이동할 수 있는 위치로 재귀적으로 수행
    for next_x, next_y in positions:
        dfs(array, next_x, next_y, total)

# 청소년 상어의 시작 위치(0, 0)에서부터 재귀적으로 모든 경우 탐색
dfs(array, 0, 0, 0)
print(result)

# 정답 확인 후 생각

In [50]:
""""""""""
답지의 전체적인 흐름과 앞부분은 내 풀이와 비슷하지만, 자잘한 디테일이나 마지막에 dfs를 수행하는 방법 등에 있어서 차이가 있다.

답지는 dfs를 수행한 후 다시 결과값을 원상복귀시키는 과정 없이 재귀적으로 구현하였다.
""""""""""

SyntaxError: EOF while scanning triple-quoted string literal (Temp/ipykernel_22824/886353925.py, line 1)